# Анализ данных сервиса для чтения книг по подписке

### Цель исследования

проанализировать базу данных — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Сформулировать ценностное предложение для нового продукта.

### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.


In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})



# Таблицы

## Таблица books

In [2]:
query = ''' SELECT *
            FROM books
        '''
books=pd.io.sql.read_sql(query, con = engine)

books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Содержит данные о книгах

## **Таблица `authors`**


In [3]:
query = ''' SELECT *
            FROM authors
        '''
authors=pd.io.sql.read_sql(query, con = engine)

authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



Содержит данные об авторах

## **Таблица `publishers`**


In [4]:
query = ''' SELECT *
            FROM publishers
        '''
publishers=pd.io.sql.read_sql(query, con = engine)

publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Содержит данные об издательствах

## **Таблица `ratings`**



In [5]:
query = ''' SELECT *
            FROM ratings
        '''
ratings=pd.io.sql.read_sql(query, con = engine)

ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Содержит данные о пользовательских оценках книг

## **Таблица `reviews`**


In [6]:
query = ''' SELECT *
            FROM reviews
        '''
reviews=pd.io.sql.read_sql(query, con = engine)

reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Содержит данные о пользовательских обзорах на книги

### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [7]:
query_1 = '''
        SELECT COUNT(*) AS books_after_01_01_20
            
        FROM 
            books
            
        WHERE
            publication_date > '2000-01-01'
        '''
query_1 = pd.io.sql.read_sql(query_1, con = engine)

query_1

,books_after_01_01_20
0,819


819 книг вышло после 1 января 2000 года

### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [8]:
query_2 = '''
        SELECT 
            b.book_id,
            b.title,
            COUNT(DISTINCT reviews.review_id) AS reviews_count, 
            ROUND(AVG(ratings.rating), 2) AS av_ratings
             
        FROM 
            books b
        LEFT JOIN reviews ON reviews.book_id = b.book_id 
        LEFT JOIN ratings ON ratings.book_id = b.book_id
            
        GROUP BY 
            b.book_id
            
        ORDER BY 
            reviews_count DESC,
            av_ratings DESC         
                        
               '''
query_2 = pd.io.sql.read_sql(query_2, con = engine)

query_2

,book_id,title,reviews_count,av_ratings
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Больше всего обзоров у Twilight (Twilight #1) -7шт	- средняя оценка 3,66

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц

In [9]:
query_4 = '''
        SELECT 
            p.publisher_id,
            p.publisher,
            COUNT(b.book_id) AS books_total
            
             
        FROM 
            books b
        LEFT JOIN publishers p ON p.publisher_id = b.publisher_id 
        
        WHERE
            b.num_pages >= 50
        
        GROUP BY
            p.publisher_id,
            p.publisher
            
        ORDER BY
            books_total DESC
                       
        LIMIT 1
             
        '''
query_4 = pd.io.sql.read_sql(query_4, con = engine)

query_4

,publisher_id,publisher,books_total
0,212,Penguin Books,42


Наибольшее количество книг выпустило издательство Penguin Books - 42 книги

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [10]:
query_5 = '''
WITH rated_books AS
    (SELECT b.book_id
     FROM books b
     LEFT JOIN ratings r ON b.book_id = r.book_id
     GROUP BY b.book_id
     HAVING count(DISTINCT r.rating_id) >= 50),

     avg_ratings AS
    (SELECT a.author,
            round(avg(r.rating), 2) AS avg_rating
     FROM books b
     JOIN rated_books f ON b.book_id = f.book_id
     LEFT JOIN ratings r ON b.book_id = r.book_id
     LEFT JOIN authors a ON b.author_id = a.author_id
     GROUP BY a.author)
     
SELECT *
FROM avg_ratings a
WHERE a.avg_rating =
        (SELECT max(avg_rating)
         FROM avg_ratings)

        '''
query_5 = pd.io.sql.read_sql(query_5, con = engine)

query_5

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


автор с самой высокой средней оценкой книг -4,29 J.K. Rowling/Mary GrandPré

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [15]:
query_5  = '''
WITH users AS
    (SELECT rev.username,
            COUNT(DISTINCT rat.rating_id) AS total_ratings
     FROM reviews rev
     LEFT JOIN ratings rat ON rev.username = rat.username
     GROUP BY rev.username
     HAVING COUNT(DISTINCT rat.rating_id) > 48),

     total_reviews AS
    (SELECT r.username,
            COUNT(DISTINCT r.review_id) AS total_reviews
     FROM reviews r
     GROUP BY r.username)
     
SELECT round(avg(tr.total_reviews)) AS avg_review
FROM users u
JOIN total_reviews tr ON u.username = tr.username
'''

query_5 = pd.io.sql.read_sql(query_5, con = engine)

query_5

,avg_review
0,24.0


среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 22

# Общий вывод

- 819 книг были выпущены после 1 января 2000 года.
- Больше всего обзоров у Twilight (Twilight #1) -7 шт	- средняя оценка 3,66
- Наибольшее количество книг выпустило издательство Penguin Books - 42 книги
- автор с самой высокой средней оценкой книг -4,28 J.K. Rowling/Mary GrandPré
- Пользователи, которые поставили больше 48 оценок, в среднем каждый написали 22 обзора

необходимо поддерживать доступ самых издаваемых и обсуждаемых книг, которые приведены в исследовании.

